# Documentation du dossier `data`

Ce document décrit le contenu et l’usage des fichiers présents dans le dossier `data` du projet.

---


---

## Fichiers et description

### 1. `positionnement-dans-la-rame-2.csv`
- **Description** : Contient les positions optimales dans les rames pour atteindre la sortie ou effectuer une correspondance.
- **Colonnes principales** :
  | Colonne | Description |
  |---------|-------------|
  | `from_id` | ID de la station de départ |
  | `to_id` | ID de la station d’arrivée |
  | `line_name` | Nom ou numéro de la ligne |
  | `position_average` | Position moyenne recommandée dans la rame |

---

### 2. `metro_connexion_corresp_idfm_ref.csv`
- **Description** : Contient les descriptions textuelles des parcours à suivre pour effectuer les correspondances entre lignes dans les gares. **Données metro-connexion.org**
- **Colonnes principales** :
  | Colonne | Description |
  |---------|-------------|
  | `Station` | Nom de la station |
  | `Ligne de Provenance` | Ligne d’arrivée dans la correspondance |
  | `Ligne de Destination` | Ligne à emprunter après la correspondance |
  | `ID Zone arret ICAR` | ID de la zone d’arrêt correspondant à la section `from` |
  | `Description détaillée` | Instructions pour effectuer la correspondance |

---

### 3. `metro_connexion_sorties_idfm_ref.csv`
- **Description** : Contient les instructions pour sortir des gares à la fin d’un trajet. **Données metro-connexion.org**
- **Colonnes principales** :
  | Colonne | Description |
  |---------|-------------|
  | `Ligne de Provenance` | Ligne utilisée pour arriver à la gare |
  | `ID Zone arret ICAR` | ID de la zone d’arrêt de la station |
  | `Description` | Instructions pour la sortie de gare |

---

### 4. `resultat_acessibilite.json`
- **Description** : Contient l'export d'une requête Navitia pour un itinéraire **Denfert-Rochereau** - **Asnières** en passant par la 4, la 12 puis la L.

---

### 5. `resultat_acessibilite_complete.json`
- **Description** : Contient l'export d'une requête Navitia pour un trajet **Denfert-Rochereau** - **Asnières**. La requête ressort 6 itinéraires différents.

---

In [128]:
import json
import csv

# Fonction : `calcule_scores_section`

## Description
Cette fonction calcule les **scores d'accessibilité** pour une section d'un itinéraire de transport public ou un transfert.
Elle évalue trois dimensions principales :

1. **Accessibilité visuelle** : présence d'annonces visuelles dans les stations.
2. **Accessibilité sonore** : présence d'annonces sonores dans les stations.
3. **Accessibilité PMR** : accessibilité pour les personnes à mobilité réduite, incluant les équipements pour fauteuil roulant.

La fonction affiche également un **résumé textuel** indiquant les éventuelles non-conformités pour chaque dimension.

---

## Entrées

| Paramètre | Type | Description |
|-----------|------|-------------|
| `section` | `dict` | Un dictionnaire représentant une section du trajet. Peut être de type `public_transport`, `transfer`, ou autre (crow_fly, etc.). Contient au minimum les clés : `section_type`, `from`, `to`, `intermediate_stops`, `transport`, `transfer_equipment_type`. |

**Structure attendue pour `section`:**

- `section_type` : `"public_transport"`, `"transfer"`, `"crow_fly"`, etc.
- `from` : dictionnaire de la station de départ
  - `id`, `name`, `accessibility_level_id`, `equipments` (liste)
- `to` : dictionnaire de la station d’arrivée
  - `id`, `name`, `accessibility_level_id`, `equipments` (liste)
- `intermediate_stops` : liste de dictionnaires similaires à `from`/`to`
- `transport` : dictionnaire du transport
  - `label`, `equipments` (liste)
- `transfer_equipment_type` : type de transfert pour les sections `transfer` (`Ascenseur`, `Escalator`, `Escalier`, etc.)

---

## Méthodologie des calculs

1. **Scores visuel et sonore**
   - Chaque station (départ, intermédiaire, arrivée) reçoit un score 0 ou 1 selon la présence d’annonces visuelles ou sonores.
   - Les scores sont pondérés (départ/arrivée/intermédiaires) pour produire un score total entre 0 et 1.

2. **Score PMR**
   - Basé sur `accessibility_level_id` et la présence de `has_wheelchair_boarding` pour les stations.
   - Les lignes de transport peuvent augmenter le score si `has_wheelchair_accessibility` est présent.
   - Pour les transferts, le type de transfert influence également le score PMR.

3. **Gestion des transferts**
   - Score visuel et sonore : moyenne des stations de départ et d’arrivée.
   - Score PMR : pondération 0.25 pour chaque station + 0.5 pour le transfert.

4. **Autres sections** (crow_fly, etc.)
   - Pas de données disponibles : tous les scores sont 0.

---

## Sortie

La fonction affiche dans la console :

- Type de section
- Noms et horaires des stations de départ et d’arrivée
- Scores pour chaque dimension
- Détails des stations non conformes si le score < 1 :
  - Stations sans annonce visuelle
  - Stations sans annonce sonore
  - Accessibilité PMR limitée (stations ou ligne)
  - Type de transfert si applicable

Elle retourne également un dictionnaire résumant les scores :

```python
{
    "type": <str>,          # type de section
    "label": <str>,         # nom de la ligne ou '?'
    "score_visuel": <float>,# score visuel [0-1]
    "score_sonore": <float>,# score sonore [0-1]
    "score_PMR": <float>    # score PMR [0-1]
}


In [118]:
def calcule_scores_section(section):
    """
    Calcule les scores d'accessibilité (visuel, sonore, PMR)
    pour une section du trajet (public_transport ou transfer).
    Affiche un résumé puis les éventuels détails de non-conformité.
    """

    # === UTILITAIRES COMMUNS ===
    def score_visuel(e):
        eq = e.get("equipments", [])
        return 1 if "has_visual_announcement" in eq else 0

    def score_sonore(e):
        eq = e.get("equipments", [])
        return 1 if "has_audible_announcement" in eq else 0

    def score_station_PMR(station):
        """Score PMR combinant accessibility_level_id et has_wheelchair_boarding."""
        level_map = {"1": 0.0, "3": 0.6, "4": 0.4, "6": 1.0}
        level = station.get("accessibility_level_id")
        base_score = level_map.get(level, 0.0)
        equips = station.get("equipments", [])

        # Bonus si équipement fauteuil roulant
        if "has_wheelchair_boarding" in equips:
            if base_score == 0.0:
                score = 0.5
            elif base_score < 1.0:
                score = min(1.0, base_score + 0.2)
            else:
                score = 1.0
        else:
            score = base_score

        return round(score, 2)

    def score_ligne_PMR(section):
        equips = section.get("transport", {}).get("equipments", [])
        return 1.0 if "has_wheelchair_accessibility" in equips else 0.0

    # === MÉTADONNÉES DE LA SECTION ===
    stype = section.get("section_type")
    depart = section.get("from", {})
    arrivee = section.get("to", {})
    inters = section.get("intermediate_stops", [])
    ligne = section.get("transport", {})
    label = ligne.get("label", "?")
    from_name, to_name = depart.get("name", "?"), arrivee.get("name", "?")

    # === EN-TÊTE ===
    print("=" * 65)
    print(f"Section — Type : {stype}")
    print(f"De : {from_name} ({section.get('departure_time')})")
    print(f"À  : {to_name} ({section.get('arrival_time')})")

    # --------------------------------------------------
    # CAS 1️⃣ : SECTION PUBLIC_TRANSPORT
    # --------------------------------------------------
    if stype == "public_transport":

        # 🔧 Corrige les doublons départ/arrivée dans les intermédiaires
        id_depart, id_arrivee = depart.get("id"), arrivee.get("id")
        inters = [s for s in inters if s.get("id") not in {id_depart, id_arrivee}]

        # Pondérations visuel/sonore
        poids_depart, poids_arrivee, poids_inter = 0.4, 0.4, 0.3
        if not inters:
            poids_depart, poids_arrivee, poids_inter = 0.5, 0.5, 0.0

        # Normalisation
        somme_poids = poids_depart + poids_arrivee + poids_inter
        poids_depart /= somme_poids
        poids_arrivee /= somme_poids
        poids_inter /= somme_poids

        # Calcul visuel/sonore
        if inters:
            visuels_inter = [score_visuel(s) for s in inters]
            sonores_inter = [score_sonore(s) for s in inters]
            score_visuel_inters = sum(visuels_inter) / len(visuels_inter)
            score_sonore_inters = sum(sonores_inter) / len(visuels_inter)
        else:
            score_visuel_inters = score_sonore_inters = 0

        score_visuel_total = (
            score_visuel(depart) * poids_depart +
            score_visuel(arrivee) * poids_arrivee +
            score_visuel_inters * poids_inter
        )

        score_sonore_total = (
            score_sonore(depart) * poids_depart +
            score_sonore(arrivee) * poids_arrivee +
            score_sonore_inters * poids_inter
        )

        # --- PMR ---
        poids_PMR = {"depart": 0.25, "arrivee": 0.25, "ligne": 0.25, "inters": 0.25}
        score_depart_PMR = score_station_PMR(depart)
        score_arrivee_PMR = score_station_PMR(arrivee)
        score_ligne_PMR_val = score_ligne_PMR(section)
        if inters:
            inters_scores = [score_station_PMR(s) for s in inters]
            score_inters_PMR = sum(inters_scores) / len(inters_scores)
        else:
            score_inters_PMR = 0
        score_PMR_total = (
            score_depart_PMR * poids_PMR["depart"] +
            score_arrivee_PMR * poids_PMR["arrivee"] +
            score_ligne_PMR_val * poids_PMR["ligne"] +
            score_inters_PMR * poids_PMR["inters"]
        )

    # --------------------------------------------------
    # CAS 2️⃣ : SECTION TRANSFER
    # --------------------------------------------------
    elif stype == "transfer":

        # VISUEL / SONORE : 50% départ + 50% arrivée
        score_visuel_total = (
            (score_visuel(depart) + score_visuel(arrivee)) / 2
        )
        score_sonore_total = (
            (score_sonore(depart) + score_sonore(arrivee)) / 2
        )

        # PMR : 0.25 départ + 0.25 arrivée + 0.5 transfert
        type_transfert = section.get("transfer_equipment_type", "")
        score_transfert_map = {
            "Ascenseur": 1.0,
            "Escalator": 0.3,
            "Escalier": 0.0,
            "": 0.0,
            None: 0.0
        }
        score_transfert = score_transfert_map.get(type_transfert, 0.0)

        score_PMR_total = (
            0.25 * score_station_PMR(depart) +
            0.25 * score_station_PMR(arrivee) +
            0.5 * score_transfert
        )

        print(f"Type de transfert : {type_transfert or 'non précisé'}")

    # --------------------------------------------------
    # CAS AUTRES (crow_fly, etc.)
    # --------------------------------------------------
    else:
        # On attribue 0 par défaut (pas d’infos)
        score_visuel_total = score_sonore_total = score_PMR_total = 0.0

    # === AFFICHAGE DES SCORES ===
    print(f"  🔹 Score visuel : {score_visuel_total:.2f}")
    print(f"  🔹 Score sonore : {score_sonore_total:.2f}")
    print(f"  ♿  Score PMR    : {score_PMR_total:.2f}")

    # === DÉTAILS SI PARTIELS ===
    if score_visuel_total < 1:
        print("  ⚠️  Stations sans annonce visuelle :")
        for stop in [depart, arrivee]:
            if "has_visual_announcement" not in stop.get("equipments", []):
                print(f"     - {stop.get('name', '?')}")

    if score_sonore_total < 1:
        print("  ⚠️  Stations sans annonce sonore :")
        for stop in [depart, arrivee]:
            if "has_audible_announcement" not in stop.get("equipments", []):
                print(f"     - {stop.get('name', '?')}")

    if score_PMR_total < 1:
        print("  ♿  Accessibilité limitée :")
        # Cas des stations
        for stop in [depart, arrivee]:
            s_id = stop.get("accessibility_level_id")
            s_name = stop.get("name", "?")
            s_score = score_station_PMR(stop)
            if s_score < 1:
                if not s_id or s_id == "":
                    print(f"     - {s_name} (station inaccessible)")
                else:
                    print(f"     - {s_name} (niveau {s_id})")
        # Cas du transfert
        if stype == "transfer" and score_transfert < 1:
            print(f"     - Type de transfert : {type_transfert or 'non précisé'} (non adapté PMR)")

        # Cas de la ligne
        if stype == "public_transport" and score_ligne_PMR_val < 1:
            print(f"     - Ligne {label} non totalement accessible")

    print()  # ligne vide
    return {
        "type": stype,
        "label": label,
        "score_visuel": round(score_visuel_total, 2),
        "score_sonore": round(score_sonore_total, 2),
        "score_PMR": round(score_PMR_total, 2)
    }


In [119]:
with open("data/resultat_accessibilite.json", "r", encoding="utf-8") as f:
    data = json.load(f)

for section in data:
    calcule_scores_section(section)

Section — Type : crow_fly
De : Denfert-Rochereau (Paris) (14:29)
À  : Denfert-Rochereau (14:29)
  🔹 Score visuel : 0.00
  🔹 Score sonore : 0.00
  ♿  Score PMR    : 0.00
  ⚠️  Stations sans annonce visuelle :
     - Denfert-Rochereau (Paris)
  ⚠️  Stations sans annonce sonore :
     - Denfert-Rochereau (Paris)
  ♿  Accessibilité limitée :
     - Denfert-Rochereau (Paris) (station inaccessible)
     - Denfert-Rochereau (station inaccessible)

Section — Type : public_transport
De : Denfert-Rochereau (14:29)
À  : Montparnasse Bienvenue (14:32)
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Denfert-Rochereau (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Ligne 4 non totalement accessible

Section — Type : transfer
De : Montparnasse Bienvenue (14:32)
À  : Montparnasse Bienvenue (14:33)
Type de transfert : non précisé
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  A

# Fonction : `calcule_score_global`

## Description
Cette fonction calcule les **scores moyens d'accessibilité** pour l'ensemble d'un itinéraire de transport public.
Les scores sont calculés sur trois dimensions principales :

1. **Accessibilité visuelle** : moyenne des scores visuels des sections.
2. **Accessibilité sonore** : moyenne des scores sonores des sections.
3. **Accessibilité PMR** : moyenne des scores PMR des sections.

Les sections de type `crow_fly` (trajets à pied simplifiés) sont **exclues** du calcul.
La fonction affiche également la séquence des lignes empruntées (ex : `4 → 12 → L`).

---

## Entrées

| Paramètre | Type | Description |
|-----------|------|-------------|
| `itineraire` | `list` | Liste de sections du trajet, où chaque section est un dictionnaire compatible avec `calcule_scores_section`. |

---

## Méthodologie des calculs

1. Pour chaque section non `crow_fly`, la fonction appelle `calcule_scores_section(section)` pour obtenir les scores individuels.
2. Elle calcule ensuite la moyenne des scores visuel, sonore et PMR sur toutes les sections valides.
3. Les labels des lignes de transport empruntées sont extraits et concaténés dans l'ordre pour former un parcours global.

---

## Sortie

La fonction affiche dans la console :

- Nombre de sections valides prises en compte
- Séquence des lignes empruntées (ex : `4 → 12 → L`)
- Scores moyens pour chaque dimension : visuel, sonore, PMR

Elle retourne également un dictionnaire récapitulatif :

```python
{
    "sections_comptées": <int>,          # nombre de sections utilisées pour le calcul
    "lignes_empruntées": <list[str]>,   # liste des labels de lignes empruntées
    "score_visuel_moyen": <float>,      # moyenne des scores visuel [0-1]
    "score_sonore_moyen": <float>,      # moyenne des scores sonore [0-1]
    "score_PMR_moyen": <float>          # moyenne des scores PMR [0-1]
}


In [120]:
def calcule_score_global(itineraire):
    """
    Calcule les scores moyens d'accessibilité (visuel, sonore, PMR)
    sur l'ensemble des sections d'un itinéraire.
    Exclut les sections de type 'crow_fly'.
    Affiche également la séquence des lignes empruntées (ex : 4 → 12 → L).
    """

    scores = []
    lignes = []  # stocke les labels de lignes empruntées dans l'ordre

    for section in itineraire:
        stype = section.get("section_type")

        # On ignore les trajets à pied simplifiés
        if stype == "crow_fly":
            continue

        # Calcul des scores
        result = calcule_scores_section(section)
        scores.append(result)

        # Récupération de la ligne pour affichage du parcours
        if stype == "public_transport":
            label = section.get("transport", {}).get("label")
            if label:
                lignes.append(str(label))  # ex : "4" ou "12" ou "L"

    if not scores:
        print("⚠️ Aucun segment valide pour le calcul.")
        return None

    # Moyenne des scores
    n = len(scores)
    moy_visuel = sum(s["score_visuel"] for s in scores) / n
    moy_sonore = sum(s["score_sonore"] for s in scores) / n
    moy_PMR = sum(s["score_PMR"] for s in scores) / n

    # Prépare la chaîne des lignes empruntées
    parcours_lignes = " → ".join(lignes) if lignes else "Aucune ligne"

    # Résumé global
    print("=" * 65)
    print("🏁 RÉSUMÉ GLOBAL DE L’ITINÉRAIRE")
    print(f"Nombre de sections prises en compte : {n}")
    print(f"Lignes empruntées : {parcours_lignes}")
    print(f"🔹 Score visuel moyen : {moy_visuel:.2f}")
    print(f"🔹 Score sonore moyen : {moy_sonore:.2f}")
    print(f"♿  Score PMR moyen    : {moy_PMR:.2f}")
    print("=" * 65)
    print()

    return {
        "sections_comptées": n,
        "lignes_empruntées": lignes,
        "score_visuel_moyen": round(moy_visuel, 2),
        "score_sonore_moyen": round(moy_sonore, 2),
        "score_PMR_moyen": round(moy_PMR, 2)
    }


In [121]:
calcule_score_global(data)

Section — Type : public_transport
De : Denfert-Rochereau (14:29)
À  : Montparnasse Bienvenue (14:32)
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Denfert-Rochereau (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Ligne 4 non totalement accessible

Section — Type : transfer
De : Montparnasse Bienvenue (14:32)
À  : Montparnasse Bienvenue (14:33)
Type de transfert : non précisé
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Montparnasse Bienvenue (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Type de transfert : non précisé (non adapté PMR)

Section — Type : public_transport
De : Montparnasse Bienvenue (14:36)
À  : Saint-Lazare (14:48)
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Montparnasse Bienvenue (station inaccessible)
     - S

{'sections_comptées': 5,
 'lignes_empruntées': ['4', '12', 'L'],
 'score_visuel_moyen': 0.7,
 'score_sonore_moyen': 0.7,
 'score_PMR_moyen': 0.31}

# Fonction : `calcule_scores_tous_itineraires`

## Description
Cette fonction calcule et affiche les **scores globaux d'accessibilité** pour **chaque itinéraire** d'un ensemble de trajets.
Les sections de type `crow_fly` (trajets à pied simplifiés) sont **exclues** du calcul.

Pour chaque itinéraire, la fonction affiche :

- Les scores moyens d'accessibilité : visuel, sonore, PMR
- La séquence des lignes empruntées (ex : `4 → 12 → L`)
- Le nombre de correspondances et la durée du trajet
- Les informations de départ et d'arrivée

Elle retourne également une liste de dictionnaires contenant les résultats pour chaque itinéraire.

---

## Entrées

| Paramètre | Type | Description |
|-----------|------|-------------|
| `data` | `list` | Liste d'itinéraires, où chaque élément est un dictionnaire représentant un itinéraire. Chaque itinéraire doit contenir au moins les clés : `journey_id`, `sections`, `nb_transfers`, `departure_time`, `arrival_time`, `duration`. |

---

## Méthodologie

1. Boucle sur tous les itinéraires de `data`.
2. Pour chaque itinéraire :
   - Récupération des informations générales : ID, nombre de correspondances, durée, heure de départ et d'arrivée.
   - Appel de la fonction `calcule_score_global` sur les sections de l'itinéraire pour obtenir les scores moyens.
   - Affichage des scores et de la séquence des lignes empruntées.
   - Stockage des résultats dans un dictionnaire pour export ou traitement ultérieur.

---

## Sortie

Retourne une liste de dictionnaires, un pour chaque itinéraire, contenant les clés suivantes :

```python
[
    {
        "itineraire": <int>,                  # Numéro de l’itinéraire dans la liste
        "journey_id": <str>,                  # ID de l’itinéraire
        "depart": <str>,                      # Heure de départ
        "arrivee": <str>,                     # Heure d’arrivée
        "nb_transfers": <int>,                # Nombre de correspondances
        "lignes": <list[str]>,                # Liste des labels de lignes empruntées
        "score_visuel": <float>,              # Score visuel moyen [0-1]
        "score_sonore": <float>,              # Score sonore moyen [0-1]
        "score_PMR": <float>                  # Score PMR moyen [0-1]
    },
    ...
]


In [123]:
def calcule_scores_tous_itineraires(data):
    """
    Calcule et affiche les scores globaux pour chaque itinéraire d'un ensemble.
    Exclut les sections 'crow_fly'.
    Affiche pour chaque itinéraire :
      - les scores (visuel / sonore / PMR)
      - les lignes empruntées (ex: 4 → 12 → L)
      - le nombre de correspondances
    """

    print("\n================= CALCUL GLOBAL TOUS ITINÉRAIRES =================\n")

    resultats = []

    for i, itin in enumerate(data, start=1):
        journey_id = itin.get("journey_id", f"itinéraire_{i}")
        nb_transfers = itin.get("nb_transfers", 0)
        duree = itin.get("duration", "N/A")
        dep = itin.get("departure_time", "?")
        arr = itin.get("arrival_time", "?")

        print(f"🚌 Itinéraire {i} — ID : {journey_id}")
        print(f"Départ : {dep} → Arrivée : {arr}")
        print(f"Durée : {duree} sec | Correspondances : {nb_transfers}")
        print("-" * 65)

        # Calcul des scores globaux
        scores = calcule_score_global(itin["sections"])

        # Récupération des lignes empruntées
        lignes = scores.get("lignes_empruntées", [])
        lignes_str = " → ".join(lignes) if lignes else "Aucune ligne"

        print(f"🚇 Lignes empruntées : {lignes_str}")
        print(f"🔹 Score visuel moyen : {scores['score_visuel_moyen']}")
        print(f"🔹 Score sonore moyen : {scores['score_sonore_moyen']}")
        print(f"♿  Score PMR moyen    : {scores['score_PMR_moyen']}")
        print("=" * 65 + "\n")

        # Stockage pour un éventuel export
        resultats.append({
            "itineraire": i,
            "journey_id": journey_id,
            "depart": dep,
            "arrivee": arr,
            "nb_transfers": nb_transfers,
            "lignes": lignes,
            "score_visuel": round(scores["score_visuel_moyen"], 2),
            "score_sonore": round(scores["score_sonore_moyen"], 2),
            "score_PMR": round(scores["score_PMR_moyen"], 2),
        })

    print("✅ Calcul terminé pour tous les itinéraires.\n")
    return resultats


In [124]:
with open("data/resultat_accessibilite-complete.json", "r", encoding="utf-8") as f:
    data = json.load(f)

calcule_scores_tous_itineraires(data)


================= CALCUL GLOBAL TOUS ITINÉRAIRES =================

🚌 Itinéraire 1 — ID : 1
Départ : 17:31 → Arrivée : 18:04
Durée : 2020 sec | Correspondances : 2
-----------------------------------------------------------------
Section — Type : public_transport
De : Denfert-Rochereau (17:31)
À  : Montparnasse Bienvenue (17:34)
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Denfert-Rochereau (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Ligne 4 non totalement accessible

Section — Type : transfer
De : Montparnasse Bienvenue (17:34)
À  : Montparnasse Bienvenue (17:35)
Type de transfert : non précisé
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Montparnasse Bienvenue (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Type de transfert : non précisé (non adapté PMR)

Section — Type : public_tran

[{'itineraire': 1,
  'journey_id': 1,
  'depart': '17:31',
  'arrivee': '18:04',
  'nb_transfers': 2,
  'lignes': ['4', '12', 'L'],
  'score_visuel': 0.7,
  'score_sonore': 0.7,
  'score_PMR': 0.31},
 {'itineraire': 2,
  'journey_id': 2,
  'depart': '17:35',
  'arrivee': '18:15',
  'nb_transfers': 1,
  'lignes': ['13', 'J'],
  'score_visuel': 0.38,
  'score_sonore': 0.38,
  'score_PMR': 0.21},
 {'itineraire': 3,
  'journey_id': 3,
  'depart': '17:33',
  'arrivee': '18:15',
  'nb_transfers': 2,
  'lignes': ['B', 'A', 'L'],
  'score_visuel': 0.0,
  'score_sonore': 0.0,
  'score_PMR': 0.58},
 {'itineraire': 4,
  'journey_id': 4,
  'depart': '17:32',
  'arrivee': '18:15',
  'nb_transfers': 2,
  'lignes': ['6', '1', 'L'],
  'score_visuel': 0.7,
  'score_sonore': 0.7,
  'score_PMR': 0.17},
 {'itineraire': 5,
  'journey_id': 5,
  'depart': '17:31',
  'arrivee': '18:15',
  'nb_transfers': 2,
  'lignes': ['4', 'A', 'L'],
  'score_visuel': 0.38,
  'score_sonore': 0.38,
  'score_PMR': 0.31},
 {'i

# Fonction : `affiche_scores_itineraires`

## Description
Cette fonction affiche **de manière lisible et synthétique** les scores globaux d’accessibilité de plusieurs itinéraires.
Elle est conçue pour fonctionner avec la sortie renvoyée par la fonction `calcule_scores_tous_itineraires()`.

Pour chaque itinéraire, elle affiche dans un tableau les informations suivantes :

- Numéro de l’itinéraire
- Lignes empruntées (ex : `4 → 12 → L`)
- Nombre de correspondances
- Score visuel moyen
- Score sonore moyen
- Score PMR moyen

---

## Entrées

| Paramètre | Type | Description |
|-----------|------|-------------|
| `resultats` | `list` | Liste de dictionnaires, chacun représentant un itinéraire. La structure attendue correspond à celle renvoyée par `calcule_scores_tous_itineraires()`. |

---

## Méthodologie

1. Vérifie si la liste `resultats` n’est pas vide.
   - Si vide, affiche un message d’avertissement et quitte la fonction.
2. Affiche un en-tête clair pour la synthèse.
3. Affiche un tableau avec une ligne par itinéraire :
   - Numéro de l’itinéraire
   - Lignes empruntées (séparées par `→`)
   - Nombre de correspondances
   - Scores moyens : visuel, sonore, PMR
4. Termine l’affichage par une ligne de séparation.

---

## Sortie

- Affiche directement le tableau sur la console.
- **Ne retourne rien** (`None`).

---

## Exemple d’utilisation

```python
# Supposons que `resultats` est le retour de calcule_scores_tous_itineraires()
affiche_scores_itineraires(resultats)


In [126]:
def affiche_scores_itineraires(resultats):
    """
    Affiche joliment les scores globaux de plusieurs itinéraires.
    Attendu : la liste de dictionnaires renvoyée par calcule_scores_tous_itineraires().
    """

    if not resultats:
        print("⚠️ Aucun résultat à afficher.")
        return

    print("\n=================== SYNTHÈSE DES ITINÉRAIRES ===================\n")
    header = f"{'Itin.':<6} {'Lignes empruntées':<25} {'Transferts':<10} {'Visuel':<8} {'Sonore':<8} {'PMR':<8}"
    print(header)
    print("-" * len(header))

    for r in resultats:
        lignes = " → ".join(r["lignes"]) if r["lignes"] else "-"
        print(f"{r['itineraire']:<6} {lignes:<25} {r['nb_transfers']:<10} {r['score_visuel']:<8.2f} {r['score_sonore']:<8.2f} {r['score_PMR']:<8.2f}")

    print("\n=================================================================\n")


In [127]:
affiche_scores_itineraires(calcule_scores_tous_itineraires(data))


================= CALCUL GLOBAL TOUS ITINÉRAIRES =================

🚌 Itinéraire 1 — ID : 1
Départ : 17:31 → Arrivée : 18:04
Durée : 2020 sec | Correspondances : 2
-----------------------------------------------------------------
Section — Type : public_transport
De : Denfert-Rochereau (17:31)
À  : Montparnasse Bienvenue (17:34)
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Denfert-Rochereau (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Ligne 4 non totalement accessible

Section — Type : transfer
De : Montparnasse Bienvenue (17:34)
À  : Montparnasse Bienvenue (17:35)
Type de transfert : non précisé
  🔹 Score visuel : 1.00
  🔹 Score sonore : 1.00
  ♿  Score PMR    : 0.00
  ♿  Accessibilité limitée :
     - Montparnasse Bienvenue (station inaccessible)
     - Montparnasse Bienvenue (station inaccessible)
     - Type de transfert : non précisé (non adapté PMR)

Section — Type : public_tran

# Fonction : `affiche_accessibilite_complet`

## Description
Cette fonction affiche de manière détaillée l’**accessibilité d’UN itinéraire complet**, section par section.
Elle gère différents types de sections (`public_transport`, `transfer`) et fournit des informations sur :

- Les **stations de départ et d’arrivée** : accessibilité visuelle, sonore et PMR.
- Les **stations intermédiaires** empruntées sur une ligne.
- Le **meilleur emplacement dans la rame** pour faciliter la correspondance suivante.
- Les **équipements disponibles sur la ligne** (fauteuil roulant, climatisation).
- Les **transferts** : type de transfert, informations PMR et description textuelle du chemin.
- La **sortie de gare** pour la dernière section de transport en commun.

Les descriptions textuelles proviennent des jeux de données **metro-connexion.org**.

---

## Entrées

| Paramètre | Type | Description |
|-----------|------|-------------|
| `itineraire` | `list` | Liste de sections d’un itinéraire. Chaque section est un dictionnaire contenant les clés `section_type`, `from`, `to`, `transport`, `intermediate_stops`, etc. |

---

## Fonctionnement

1. **Parcours de chaque section de l’itinéraire**.
2. **Pour les sections `public_transport` :**
   - Affiche la station de départ avec :
     - Accessibilité visuelle et sonore.
     - Texte PMR basé sur `accessibility_level_id`.
     - Position optimale dans la rame pour la correspondance suivante.
   - Affiche le transport utilisé et ses équipements.
   - Liste les stations intermédiaires empruntées.
   - Affiche la station d’arrivée avec les mêmes informations.
   - Si c’est la dernière section, affiche également une description pour la **sortie de gare**.
3. **Pour les sections `transfer` :**
   - Détermine les lignes de provenance et de destination.
   - Affiche le type de transfert avec un texte explicatif PMR.
   - Fournit la **description textuelle du chemin** à suivre pour effectuer la correspondance.
4. Utilise plusieurs fichiers CSV pour récupérer :
   - La **position optimale** dans la rame (`positionnement-dans-la-rame-2.csv`).
   - La **description du chemin pour les correspondances** (`metro_connexion_corresp_idfm_ref.csv`).
   - La **description pour la sortie de gare** (`metro_connexion_sorties_idfm_ref.csv`).

---

## Fonctions auxiliaires utilisées

- `clean_stop_id(stop_id)` : récupère la partie pertinente d’un ID de station.
- `get_position_average(from_id, to_id, line_name)` : récupère la position optimale pour la correspondance.
- `texte_accessibility_level(level)` : convertit `accessibility_level_id` en texte explicatif.
- `texte_position(position)` : affiche le texte du meilleur emplacement dans la rame.
- `texte_equipements_ligne(equips)` : affiche les équipements disponibles sur la ligne.
- `texte_type_transfer(transfer_type)` : fournit un texte explicatif sur le type de transfert.
- `get_transfer_description(ligne_entree, ligne_sortie, from_station_id)` : récupère la description textuelle de la correspondance.
- `get_sortie_description(line, to_station_id)` : récupère la description de sortie de gare.

---

## Exemple d’utilisation

```python
# Affiche l’accessibilité complète d’un itinéraire JSON
affiche_accessibilite_complet(itineraire)


In [114]:
def clean_stop_id(stop_id):
    """Récupère la partie avant le dernier ':'"""
    return stop_id.rsplit(":", 1)[-1]

def get_position_average(from_id, to_id, line_name, csv_path="data/positionnement-dans-la-rame-2.csv"):
    """Récupère la position optimale depuis le CSV"""
    from_id_clean = clean_stop_id(from_id)
    to_id_clean = clean_stop_id(to_id)
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter=";")
        for row in reader:
            if row["from_id"] == from_id_clean and row["to_id"] == to_id_clean and row["line_name"] == line_name:
                return row.get("position_average", None)
    return None

def texte_accessibility_level(level):
    mapping = {
        "1": "La gare n'est pas accessible PMR",
        "3": "Il est possible de réserver une assistance en gare via le service Assist'en'Gare. Ce service doit être commandé 24h à l'avance",
        "4": "Il est possible de demander une assistance en gare",
        "6": "La gare est accessible en toute autonomie"
    }
    if level in mapping:
        return mapping[level]
    return "Pas d'informations d'accessibilité sur la gare"

def texte_position(position):
    if position:
        return f"Position pour faciliter la correspondance : {position}"
    else:
        return "Impossible de déterminer une position préférentielle pour la correspondance"

def texte_equipements_ligne(equips):
    has_wheelchair = "has_wheelchair_accessibility" in equips
    has_clim = "has_air_conditioned" in equips

    if has_wheelchair and has_clim:
        return "Accessible au fauteuil roulant et présence de climatisation"
    elif has_wheelchair:
        return "Accessible au fauteuil roulant"
    elif has_clim:
        return "Présence de climatisation"
    else:
        return "Pas d'information sur l'accessibilité UFR ou la présence de climatisation"

def texte_type_transfer(transfer_type):
    if not transfer_type:
        return "Pas d'information disponible sur l'accessibilité PMR de la correspondance"
    if transfer_type.lower() == "ascenseur":
        return "La correspondance est possible pour les usagers en fauteuil roulant"
    elif transfer_type.lower() == "escalator":
        return "Attention, la correspondance nécessite une assistance en gare"
    elif transfer_type.lower() == "escalier":
        return "Attention, la correspondance n'est pas directement accessible pour les usagers en fauteuil roulant. Faites appel à l'assistance"
    else:
        return f"Présence de {transfer_type}"

def get_transfer_description(ligne_entree, ligne_sortie, from_station_id, csv_path="data/metro_connexion_corresp_idfm_ref.csv"):
    """Récupère la description détaillée du chemin à emprunter pour le transfer"""
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if (row["Ligne de Provenance"] == ligne_entree and
                row["Ligne de Destination"] == ligne_sortie and
                str(row["ID Zone arret ICAR"]) == from_station_id):
                return row.get("Description détaillée") or "Pas d'information textuelle pour la correspondance"
    return "Pas d'information textuelle pour la correspondance"

def get_sortie_description(line, to_station_id, csv_path="data/metro_connexion_sorties_idfm_ref.csv"):
    """Récupère la description de sortie de gare pour la dernière section"""
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if str(row["Ligne de Provenance"]) == str(line) and str(row["ID Zone arret ICAR"]) == to_station_id:
                return row.get("Description") or "Pas d'information textuelle pour sortir de la gare"
    return "Pas d'information textuelle pour sortir de la gare"

def affiche_accessibilite_complet(itineraire):
    derniere_ligne = None

    for idx, section in enumerate(itineraire):
        stype = section.get("section_type")

        if stype == "public_transport":
            depart = section.get("from", {})
            arrivee = section.get("to", {})
            inters = section.get("intermediate_stops", [])
            transport = section.get("transport", {})

            label = transport.get("label", "?")
            equips = transport.get("equipments", [])

            print("=" * 70)
            print(f"SECTION {idx+1} — Transport : {label}")
            print("-" * 70)

            # === Chercher le transfer suivant pour le positionnement ===
            position_avg = None
            for j in range(idx+1, len(itineraire)):
                next_section = itineraire[j]
                if next_section.get("section_type") == "transfer":
                    from_id = next_section['from']['id']
                    to_id = next_section['to']['id']
                    position_avg = get_position_average(from_id, to_id, label)
                    break

            # === Station de départ ===
            nom = depart.get("name", "Nom inconnu")
            level_text = texte_accessibility_level(depart.get("accessibility_level_id"))
            equips_station = depart.get("equipments", [])
            visuel = "has_visual_announcement" in equips_station
            sonore = "has_audible_announcement" in equips_station
            if visuel and sonore:
                acces_info = "Accessibilité visuelle et sonore"
            elif visuel:
                acces_info = "Accessibilité visuelle"
            elif sonore:
                acces_info = "Accessibilité sonore"
            else:
                acces_info = "Aucune information d'accessibilité visuelle/sonore"

            print(f"\n📍 Station de départ : {nom}")
            print(f"   → {acces_info}")
            print(f"   → {level_text}")
            print(f"   → {texte_position(position_avg)}")

            # === Transport ===
            print("\n🚇 Transport utilisé : Ligne", label)
            print(f"   → {texte_equipements_ligne(equips)}")

            # Stations empruntées
            stations_empr = [s.get("name", "?") for s in inters]
            arrival_name = arrivee.get("name")
            if arrival_name not in stations_empr:
                stations_empr.append(arrival_name)
            seen = set()
            stations_empr = [x for x in stations_empr if not (x in seen or seen.add(x))]
            if stations_empr:
                print(f"   → Stations empruntées : {' → '.join(stations_empr)}  ({len(stations_empr)} stations)")
            else:
                print("   → Aucune station empruntée")

            # === Station d'arrivée ===
            nom_arr = arrivee.get("name", "Nom inconnu")
            level_arr_text = texte_accessibility_level(arrivee.get("accessibility_level_id"))
            equips_station_arr = arrivee.get("equipments", [])
            visuel_arr = "has_visual_announcement" in equips_station_arr
            sonore_arr = "has_audible_announcement" in equips_station_arr
            if visuel_arr and sonore_arr:
                acces_info_arr = "Accessibilité visuelle et sonore"
            elif visuel_arr:
                acces_info_arr = "Accessibilité visuelle"
            elif sonore_arr:
                acces_info_arr = "Accessibilité sonore"
            else:
                acces_info_arr = "Aucune information d'accessibilité visuelle/sonore"

            print(f"\n📍 Station d'arrivée : {nom_arr}")
            print(f"   → {acces_info_arr}")
            print(f"   → {level_arr_text}")

            # === Sortie gare si dernière section ===
            if idx == len(itineraire)-1 or not any(s.get("section_type") == "public_transport" for s in itineraire[idx+1:]):
                to_station_id_clean = clean_stop_id(arrivee.get("id", ""))
                sortie_description = get_sortie_description(label, to_station_id_clean)
                print(f"   → Sortie de gare : {sortie_description}")

            print()
            derniere_ligne = label

        elif stype == "transfer":
            ligne_entree = derniere_ligne or "?"
            ligne_sortie = "?"
            for j in range(idx+1, len(itineraire)):
                if itineraire[j].get("section_type") == "public_transport":
                    ligne_sortie = itineraire[j].get("transport", {}).get("label", "?")
                    break

            transfer_type = section.get("transfer_equipment_type", "")
            transfer_text = texte_type_transfer(transfer_type)

            # === Récupération du texte détaillé du transfer ===
            from_station_id = clean_stop_id(section['from']['id'])
            description_transfer = get_transfer_description(ligne_entree, ligne_sortie, from_station_id)

            print("=" * 70)
            print(f"SECTION {idx+1} — TRANSFER")
            print("-" * 70)
            print("\n🔁 Correspondance :")
            print(f"   • Correspondance entre les lignes {ligne_entree} → {ligne_sortie}")
            print(f"   • {transfer_text}")
            print(f"   • Description : {description_transfer}")
            print()


In [115]:
with open("data/resultat_accessibilite.json") as json_file:
    json_data = json.load(json_file)
    #print(json_data)

affiche_accessibilite_complet(json_data)

SECTION 2 — Transport : 4
----------------------------------------------------------------------

📍 Station de départ : Denfert-Rochereau
   → Accessibilité visuelle et sonore
   → Pas d'informations d'accessibilité sur la gare
   → Position pour faciliter la correspondance : Avant

🚇 Transport utilisé : Ligne 4
   → Pas d'information sur l'accessibilité UFR ou la présence de climatisation
   → Stations empruntées : Denfert-Rochereau → Raspail → Vavin → Montparnasse Bienvenue  (4 stations)

📍 Station d'arrivée : Montparnasse Bienvenue
   → Accessibilité visuelle et sonore
   → Pas d'informations d'accessibilité sur la gare

SECTION 3 — TRANSFER
----------------------------------------------------------------------

🔁 Correspondance :
   • Correspondance entre les lignes 4 → 12
   • Pas d'information disponible sur l'accessibilité PMR de la correspondance
   • Description : Pas d'information textuelle pour la correspondance

SECTION 4 — Transport : 12
-----------------------------------